In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 12

In [ ]:
def get_positions(row, chosen_sweeps, min_distance=0, end_distance=7):
    
    # the end positions is more robust than the starting position.
    second_half = row.positions[row.positions.shape[0]//2:, :]
    
    try:
        end_position = second_half[second_half[:, 2] < 0.1, :][0, :2]
    except:
        print('did not detect end position with height. did drone land?')
        end_position = row.positions[-1, :2]
        print('using:', end_position)
    
    if row.appendix in ["", "_30"]:
        END_POS = np.array([0.0, 0.3])
    elif row.appendix in ["_50"]:
        print("using 0.5")
        END_POS = np.array([0.0, 0.5])
    else: # crashed into wall
        END_POS = np.array([0.0, end_distance * 1e-2])
        
    delta = end_position + END_POS
    positions_cm = np.c_[delta[0] - row.positions[chosen_sweeps, 0],
                         delta[1] - row.positions[chosen_sweeps, 1], 
                         row.positions[chosen_sweeps, 2]] * 1e2
    
    if np.any(positions_cm[:, 2] < 30):
        return None
    
    mask_ok = positions_cm[:, 1] > min_distance
    chosen_sweeps = chosen_sweeps[mask_ok]
    positions_cm = positions_cm[mask_ok, :]
    return positions_cm

In [ ]:
def get_calib_function(calib="stepper"):
    from calibration import get_calibration_function_median
    from calibration import get_calibration_function_moving
    
    if calib == "moving":
        print("using moving")
        calib_function_median, freqs = get_calibration_function_moving(
            "2021_07_27_manual", motors="all45000", fit_one_gain=False
        )
    elif calib == "stepper":
        print("using stepper dataset")
        calib_function_median, freqs = get_calibration_function_median(
            "2021_07_08_stepper_fast",
            motors="all45000",
            mic_type="audio_deck",
            snr=5,
            fit_one_gain=False,
        )
    return calib_function_median

# Calibration study

In [ ]:
from calibration import get_calibration_function_median
from plotting_tools import save_fig

XLIM = [3000, 4500]
YLIM = [3, 50]
SIZE = [5, 5]

for motors in [0, "all45000"]:
    flag = "no" if motors == 0 else ""

    fig, ax = plt.subplots()
    fig.set_size_inches(*SIZE)
    calib_function_median, freqs = get_calibration_function_median(
        "2021_07_08_stepper_fast", mic_type="audio_deck", snr=5, motors=motors, ax=ax
    )
    ax.set_title(f"stepper, with motors")
    ax.legend(loc='upper left')
    ax.set_yscale("log")
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)
    save_fig(fig, f"plots/experiments/calibration_stepper_{flag}motors.pdf")

In [ ]:
from calibration import get_calibration_function_moving

from matplotlib import cm
cmap = cm.get_cmap('inferno')

exp_name = "2021_07_27_manual"
results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")

for motors in [0, "all45000"]:
    flag = "no" if motors == 0 else ""
    
    fig_total, ax_total = plt.subplots()
    fig_total.set_size_inches(*SIZE)
    get_calibration_function_moving(exp_name, motors=motors, fit_one_gain=False, ax=ax_total)
    ax_total.set_yscale("log")
    ax_total.set_ylim(*YLIM)
    ax_total.set_xlim(*XLIM)
    save_fig(fig_total, f"plots/experiments/calibration_manual_{flag}motors.pdf")
    
    rows = results_df.loc[results_df.motors==motors]
    fig, axs = plt.subplots(1, 4)
    fig.set_size_inches(15, 5)
    for mic_i in range(4):
        for i_row, row in rows.iterrows():
            
            #if np.any(np.abs(row.stft[:, mic_i, :]) > 30):
            #    print(f'mic{mic_i}{row.appendix}: problematic')
            #else:
            #    print(f'mic{mic_i}{row.appendix}: ok')
                
            for i, (stft, freq) in enumerate(zip(row.stft, row.frequencies_matrix)):
                color = cmap(i/row.stft.shape[0])
                axs[mic_i].plot(freq[freq > 0], np.abs(stft[mic_i, freq>0]), color=color, alpha=0.2)
                
        #ax_total.plot(freq[freq > 0], np.abs(stft_average[mic_i, freq>0]), color=f"C{mic_i}")
        #ax_total.grid()
        #ax_total.legend()

# 3 by 3 study

### use stepper motor data

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG_STEPPER, WALL_DISTANCE_CM_STEPPER
exp_name = "2021_07_08_stepper_fast"
distance = 40
calib = "stepper"

azimuth_deg = WALL_ANGLE_DEG_STEPPER
d_corr = distance - WALL_DISTANCE_CM_STEPPER

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
row = results_df.loc[(results_df.distance == d_corr) &
                      (results_df.bin_selection == 5) &
                      (results_df.motors == "all45000"), :].iloc[0]

label = f"stepper at {distance}"
mics = range(4) #[0, 1]

stft_list = row.stft
freqs_list = row.frequencies_matrix
pos_list = np.array([[0, distance]] * len(stft_list))
start_i = 0 # for naming only

fname = f"plots/experiments/{exp_name}_by3_distance{distance}"

### use hovering data

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

exp_name = "2021_07_27_hover"; 
distance = 30
calib = "moving"

azimuth_deg = WALL_ANGLE_DEG

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
row = results_df.loc[results_df.appendix == f"_{distance}", :].iloc[0]

label = f"hover at {distance}"
mics = [0, 1]

# choose three sweeps to plot
start_i = 3
n_sweeps = 3

#for start_i in np.arange(row.stft.shape[0]-n_sweeps)[::3]:
chosen_sweeps = np.arange(start_i, start_i+n_sweeps)
stft_list = row.stft[chosen_sweeps]
freqs_list = row.frequencies_matrix[chosen_sweeps]
pos_list = get_positions(row, chosen_sweeps) 
dist_average = np.mean([p[1] for p in pos_list])

fname = f"plots/experiments/{exp_name}_by3_distance{distance}"

In [ ]:
from estimators import DistanceEstimator
from inference import Inference, eps_normalize
from plotting_tools import save_fig
from matplotlib import cm

cmap = cm.get_cmap('inferno')
distance_range = [0.0, 80.0]

for start_i in np.arange(row.stft.shape[0]-n_sweeps-1)[::3]:
    chosen_sweeps = np.arange(start_i, start_i+n_sweeps)
    stft_list = row.stft[chosen_sweeps]
    freqs_list = row.frequencies_matrix[chosen_sweeps]
    pos_list = get_positions(row, chosen_sweeps) 
    if pos_list is None: # invalid position
        continue
    dist_average = np.mean([p[1] for p in pos_list])

    inf_machine = Inference()
    inf_machine.add_calibration_function(calib_function_median)
    inf_machine.add_geometry(distance_range, azimuth_deg)

    fig_slice, ax_slice = plt.subplots(1, len(mics))
    fig_slice.set_size_inches(3*len(mics), 3)

    fig_prob, ax_prob = plt.subplots(1, len(mics))
    fig_prob.set_size_inches(3*len(mics), 3)

    fig_pos, ax_pos = plt.subplots()
    fig_pos.set_size_inches(3, 3)
    fig_total, ax_total = plt.subplots()
    fig_total.set_size_inches(3, 3)


    #### Treat all stfts individually
    inf_machine = Inference()
    inf_machine.add_calibration_function(calib_function_median)
    inf_machine.add_geometry(distance_range, azimuth_deg)

    distance_estimator_all = DistanceEstimator()

    dist_average = np.mean([p[1] for p in pos_list])
    for i, (stft, freqs, pos) in enumerate(zip(stft_list, freqs_list, pos_list)):
        #color = cmap(i/len(stft_list))
        color = f"C{i}"

        ax_pos.scatter(pos[0], pos[1], s=20.0, marker='x', color=color)

        inf_machine.add_data(np.abs(stft), freqs)
        inf_machine.filter_out_freqs()

        distance_estimator = DistanceEstimator()

        for mic_i, mic_idx in enumerate(mics):
            amps = np.abs(stft[mic_idx, :])
            ax_slice[mic_i].plot(freqs[freqs > 0], 
                          amps[freqs > 0], 
                          color=color,ls="-")
            dist, proba, diff = inf_machine.do_inference("bayes", mic_idx)

            distance_estimator.add_distribution(diff*1e-2, proba, mic_idx)
            distance_estimator_all.add_distribution(diff*1e-2, proba, mic_idx)

            ax_prob[mic_i].plot(dist, proba, label=f'mic{mic_idx}', color=color)

            #ax_slice.plot(freqs[freqs>0], calib_function_median(freqs[freqs>0])[mic_i], 
            #            color='C0')
        dist, proba = distance_estimator.get_distance_distribution(
            method='sum',
            distances_m=1e-2*np.arange(*distance_range, step=1.0)
        )
        #ax_total.plot(dist*1e2, eps_normalize(proba, 1e-3), color=color, ls='-', 
        #              label=f'probability sweep {i}')
    dist, proba = distance_estimator_all.get_distance_distribution(
        method='sum',
        distances_m=1e-2*np.arange(*distance_range, step=1.0),
        verbose=True
    )
    ax_total.plot(dist*1e2, eps_normalize(proba, 1e-3), color='k', ls='-', 
                  label='averaged probabilities')


    #### Create average of stfts. 
    stft_array = np.array(stft_list)
    stft = np.median(stft_array, axis=0)

    inf_machine.add_data(np.abs(stft), freqs)

    freqs = freqs_list[0]
    distance_estimator = DistanceEstimator()
    for mic_i, mic_idx in enumerate(mics):
        amps = np.abs(stft[mic_idx, :])
        ax_slice[mic_i].plot(freqs[freqs > 0], 
                             amps[freqs > 0], 
                             color='k',ls="--")
        dist, proba, diff = inf_machine.do_inference("bayes", mic_idx)
        distance_estimator.add_distribution(diff*1e-2, proba, mic_idx)
        ax_prob[mic_i].plot(dist, proba, label=f'mic{mic_idx}', color='k', ls='--')
        #ax_slice.plot(freqs[freqs>0], calib_function_median(freqs[freqs>0])[mic_i], 
        #            color='C0')
    dist, proba = distance_estimator.get_distance_distribution(
        method='sum',
        distances_m=1e-2*np.arange(*distance_range, step=1.0)
    )
    ax_total.plot(dist*1e2, eps_normalize(proba, 1e-3), color='k', ls='--', 
                  label='averaged sweeps')


    #### Plot decorations
    ax_prob[0].set_ylabel('probability')
    ax_slice[0].set_ylabel('amplitude')
    for m, mic_idx in enumerate(mics):
        ax_prob[m].set_xlabel('distance [cm]')
        ax_prob[m].set_title(f'mic{m}')
        ax_prob[m].grid()

        ax_slice[m].set_xlabel('frequency [Hz]')
        ax_slice[m].set_title(f'mic{m}')
        ax_slice[m].grid()

    ax_pos.set_xlabel('x [cm]')
    ax_pos.set_ylabel('y [cm]')
    ax_pos.axis('equal')
    #ax_pos.set_title(label)
    ax_pos.grid()

    ax_total.set_xlabel('distance [cm]')
    ax_total.set_ylabel('probability')
    #ax_total.set_title(label)
    ax_total.axvline(dist_average, ls='-', color='C1', label='average distance')
    ax_total.legend(loc='upper left', bbox_to_anchor=[1.0, 1.0])
    ax_total.grid()
    save_fig(fig_total, f"{fname}_start{start_i}_n{n_sweeps}.pdf")

# Full study

In [ ]:
from crazyflie_description_py.experiments import WALL_ANGLE_DEG

calib_method = "stepper"

#exp_name = "2021_07_27_hover"; 
#appendix = "_50"; motors="hover_sweep"

#exp_name = "2021_07_14_flying"; end_distance=7
#appendix = "_19"; motors="linear_buzzer_cont";  # 19 is not good because of positioning

#exp_name = "2021_07_14_flying_hover"; end_distance=50
#appendix = "_2"; motors="multi_sweep";  

#exp_name = "2021_07_27_manual"; end_distance=15
#appendix = "_4"; motors="all45000"

azimuth_deg = WALL_ANGLE_DEG

results_df = pd.read_pickle(f"../experiments/{exp_name}/all_data.pkl")
print(results_df.motors.unique())

row = results_df.loc[(results_df.appendix == appendix) & 
                     (results_df.motors == motors), :].iloc[0]

plot_name = f"plots/experiments/{exp_name}{appendix}_motors{motors}"

In [ ]:
def plot_positions(ax, positions_cm):
    from matplotlib import cm
    
    cmap = cm.get_cmap('inferno') 
    n_labels = 3
    label = None
    step = len(positions_cm) // n_labels
    for i, p in enumerate(positions_cm):
        if i % step == 0 or (i == len(positions_cm) - 1):
            label = f'position {i}'
        ax.scatter(*p[:2], color=cmap(i / len(positions_cm)), label=label)
        label=None
    ax.legend()
    ax.plot(positions_cm[:, 0], positions_cm[:, 1], color='k', ls=':')
    ax.axis('equal')
    ax.set_xlabel('x [cm]')
    ax.set_ylabel('y [cm]')

In [ ]:
from plotting_tools import pcolorfast_custom, FIGSIZE, save_fig
from simulation import get_df_theory

min_distance = 11
chosen_sweeps = np.where(row.positions[:, 2] > 0.3)[0]

positions_cm = get_positions(row, chosen_sweeps, min_distance, end_distance=end_distance)

fig, ax = plt.subplots()
fig.set_size_inches(FIGSIZE, FIGSIZE)
plot_positions(ax, positions_cm)
save_fig(fig, plot_name + "_positions.pdf")

freqs = row.frequencies_matrix[0, :]

distances = positions_cm[:, 1]

xvalues = distances; xlabel="distance [cm]"
#xvalues = row.seconds[chosen_sweeps]; xlabel="seconds [s]"

yvalues = freqs[freqs>0]
values = np.abs(row.stft[chosen_sweeps])[..., freqs>0]

sort_idx = np.argsort(xvalues)
xvalues = xvalues[sort_idx]
values = values[sort_idx, ...]

df = get_df_theory(yvalues, np.sort(distances))

calib_function = get_calib_function(calib_method)
calib_values = calib_function(yvalues)

values /= calib_values

mic_idx = 0

step = 500
yticks = np.arange(min(yvalues)//step*step, max(yvalues)//step*step+2*step, step=step)

fig, ax = plt.subplots()
fig.set_size_inches(2*FIGSIZE, FIGSIZE)
pcolorfast_custom(ax, np.arange(len(xvalues)), yvalues, values[:, mic_idx, :].T)
ax.set_xticklabels(np.round(xvalues, 1), rotation=90)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks)
ax.set_ylabel("frequency [Hz]")
ax.set_xlabel(xlabel)

fig, ax = plt.subplots()
fig.set_size_inches(2*FIGSIZE, FIGSIZE)
pcolorfast_custom(ax, np.arange(len(xvalues)), yvalues, df[mic_idx, :, :])
ax.set_xticklabels(np.round(xvalues, 1), rotation=90)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks)
ax.set_ylabel("frequency [Hz]")
ax.set_xlabel(xlabel)

In [ ]:
from plotting_tools import add_colorbar
from inference import Inference, eps_normalize
from estimators import DistanceEstimator

mics = range(2) #[0, 1, 2, 3]
inf_machine = Inference()

distance_estimator = DistanceEstimator()
dists = distance_estimator.DISTANCES_M * 1e2

results_matrix = np.empty((values.shape[0], len(dists)))
for i in range(values.shape[0]):
    inf_machine.add_data(values[i], yvalues)
    
    inf_machine.add_geometry([0, 80], WALL_ANGLE_DEG)
    for mic_i in mics: 
        dist, prob, diff = inf_machine.do_inference("bayes", mic_i)
        distance_estimator.add_distribution(diff*1e-2, prob, mic_i)
        
    d, prob = distance_estimator.get_distance_distribution()
    results_matrix[i, :] = eps_normalize(prob, eps=1e-50)
    
fig, ax = plt.subplots()
fig.set_size_inches(2*FIGSIZE, FIGSIZE)
im = pcolorfast_custom(ax, np.arange(values.shape[0]), dists, np.log10(results_matrix.T))
ax.set_xticklabels(np.round(xvalues, 1), rotation=90)
ax.set_ylabel("estimated distance [cm]")
ax.set_xlabel("real distance [cm]")
ax.set_yticks(np.arange(min(dists), max(dists), step=10))
ax.set_yticklabels(np.arange(min(dists), max(dists), step=10).astype(int))
ax.plot(np.arange(values.shape[0])+0.5, xvalues, color="C1", label="ground truth")
ax.set_ylim(min(dists), max(dists))
ax.legend()
add_colorbar(fig, ax, im, title="log-probability")
save_fig(fig, plot_name + "_probabilities.pdf")